In [41]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
import statistics
import math
import datetime

In [2]:
def reader(file):
    csv = open(file, 'r',encoding="utf8", errors='ignore')
    data =  pd.read_csv(csv)
    return data

In [3]:
Training = reader('train_transaction.csv')
Training.shape

(590540, 394)

In [4]:
dict(Training.isna().sum())

{'TransactionID': 0,
 'isFraud': 0,
 'TransactionDT': 0,
 'TransactionAmt': 0,
 'ProductCD': 0,
 'card1': 0,
 'card2': 8933,
 'card3': 1565,
 'card4': 1577,
 'card5': 4259,
 'card6': 1571,
 'addr1': 65706,
 'addr2': 65706,
 'dist1': 352271,
 'dist2': 552913,
 'P_emaildomain': 94456,
 'R_emaildomain': 453249,
 'C1': 0,
 'C2': 0,
 'C3': 0,
 'C4': 0,
 'C5': 0,
 'C6': 0,
 'C7': 0,
 'C8': 0,
 'C9': 0,
 'C10': 0,
 'C11': 0,
 'C12': 0,
 'C13': 0,
 'C14': 0,
 'D1': 1269,
 'D2': 280797,
 'D3': 262878,
 'D4': 168922,
 'D5': 309841,
 'D6': 517353,
 'D7': 551623,
 'D8': 515614,
 'D9': 515614,
 'D10': 76022,
 'D11': 279287,
 'D12': 525823,
 'D13': 528588,
 'D14': 528353,
 'D15': 89113,
 'M1': 271100,
 'M2': 271100,
 'M3': 271100,
 'M4': 281444,
 'M5': 350482,
 'M6': 169360,
 'M7': 346265,
 'M8': 346252,
 'M9': 346252,
 'V1': 279287,
 'V2': 279287,
 'V3': 279287,
 'V4': 279287,
 'V5': 279287,
 'V6': 279287,
 'V7': 279287,
 'V8': 279287,
 'V9': 279287,
 'V10': 279287,
 'V11': 279287,
 'V12': 76073,
 

In [30]:
Training2 = Training.iloc[:,0:376]
Training2 = Training2.iloc[0:int(len(Training2)*0.03)]
num_cols = Training2._get_numeric_data().columns
cate_cols = list(set(Training2.columns) - set(num_cols))
for cols in cate_cols:
    Training2[cols] = Training2[cols].astype('category').cat.codes
Training2.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321
0,2987000,0,86400,68.5,4,13926,NaN,150.0,1,142.0,...,0.0,0.0,0.0,0.0,0.0,117.0,0.0,0.0,0.0,0.0
1,2987001,0,86401,29.0,4,2755,404.0,150.0,2,102.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2987002,0,86469,59.0,4,4663,490.0,150.0,3,166.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2987003,0,86499,50.0,4,18132,567.0,150.0,2,117.0,...,135.0,0.0,0.0,0.0,50.0,1404.0,790.0,0.0,0.0,0.0
4,2987004,0,86506,50.0,1,4497,514.0,150.0,2,102.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Filling the missing value by Mode and Mean
def Avgmiss(df):
    for columns in list(df.columns):
        #if not isinstance(df[columns][0],str):
        if columns in ['dist1','dist2','card2'] and columns[0] =='D':
            df[columns].fillna(df[columns].mean(),inplace = True)
        else:
            df[columns].fillna(df[columns].mode()[0],inplace = True)
            
    return df
#res = [Training2,Testing2]
#update = [(lambda x: Avgmiss(x))(x) for x in res]
Miss_Meth1 = Avgmiss(Training2)

In [ ]:
# Filling the missing value by Mode and RF
def rfmiss(df):
    for columns in list(df.columns):
        #if not isinstance(df[columns][0],str):
        if columns != 'dist1' and 'dist2' and 'card2' and columns[0] !='D':
            df[columns].fillna(df[columns].mode(),inplace = True)
        
    complete_cols_df = df[df.columns[df.isna().sum() == 0].tolist()]
    complete_cols_df = complete_cols_df.drop( [x for x in complete_cols_df.columns if isinstance(complete_cols_df[x][0],str)],axis = 1 )
    uncomplete_cols = df.columns[df.isna().any()].tolist()
    for col in uncomplete_cols:
        complete_cols_df[col] = df[col]
       # complete_cols_df = complete_cols_df[complete_cols_df[col].notna()]
        known_part = complete_cols_df[complete_cols_df[col].notna()].as_matrix()
        unknown_part = complete_cols_df[complete_cols_df[col].isna()].as_matrix()
        y = known_part[:, -1]
        X = known_part[:,0:complete_cols.shape[1]-2]
        rfr = RandomForestRegressor(random_state=0, n_estimators=2000, n_jobs=-1)
        rfr.fit(X, y)
        predictedAges = rfr.predict(unknown_part[:,0:complete_cols.shape[1]-2])
        df.loc[ (df[col].isna()), col ] = predictedAges
    
            
    return df
#res = [Training2,Testing2]
#update = [(lambda x: Avgmiss(x))(x) for x in res]
#Miss_Meth2 = rfmiss(Training2)
# 问题：速度太慢


In [31]:
# Filling the missing value with Mode and Multiple imputation(based on linear regression)
def multiple_linear_fit(df,replicates):
    for columns in list(df.columns):
        #if not isinstance(df[columns][0],str):
        if columns  not in ['dist1' ,'dist2' ,'card2'] and columns[0] !='D':
            df[columns].fillna(df[columns].mode()[0],inplace = True)
    complete_cols_df = df[df.columns[df.isna().sum() == 0].tolist()]
    #complete_cols_df = complete_cols_df.drop( [x for x in complete_cols_df.columns if isinstance(complete_cols_df[x][0],str)],axis = 1 )
    uncomplete_cols = df.columns[df.isna().any()].tolist()
    for col in uncomplete_cols:
        complete_cols_df[col] = df[col]
        known_part = complete_cols_df[complete_cols_df[col].notna()].as_matrix()
        unknown_part = complete_cols_df[complete_cols_df[col].isna()].as_matrix()
        y = known_part[:, -1]
        X = known_part[:,0:complete_cols_df.shape[1]-2]
       
        lrg = LinearRegression()
        lrg.fit(X, y)
        cnt = 0
        predictedAges = lrg.predict(unknown_part[:,0:complete_cols_df.shape[1]-2])
        for cnt in range(0,replicates-1):
            predictedAges += lrg.predict(unknown_part[:,0:complete_cols_df.shape[1]-2])
            cnt += 1
        df.loc[ (df[col].isna()), col ] = predictedAges/replicates
        complete_cols_df[col] = df[col]
    return df
Miss_Meth2 = multiple_linear_fit(Training2,5)

C:\Users\Alienware\Anaconda3\envs\EVM\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\Alienware\Anaconda3\envs\EVM\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':
C:\Users\Alienware\Anaconda3\envs\EVM\lib\site-packages\ipykernel_launcher.py:13: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  del sys.path[0]
C:\Users\Alienware\Anaconda3\envs\EVM\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,

C:\Users\Alienware\Anaconda3\envs\EVM\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Alienware\Anaconda3\envs\EVM\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\Alienware\Anaconda3\envs\EVM\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':
C:\Users\Alienware\Anaconda3\envs\EVM\l

C:\Users\Alienware\Anaconda3\envs\EVM\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Alienware\Anaconda3\envs\EVM\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':
C:\Users\Alienware\Anaconda3\envs\EVM\lib\site-packages\ipykernel_launcher.py:13: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  del sys.path[0]


In [42]:
# Feature selection
# Fisher Cirteria
start_time = datetime.datetime.now() 
datafinal = Training2
key = list(datafinal.columns)[:-1]
dic = {x:True for x in key}
for i in range(len(key)):
    mk = statistics.mean(datafinal[key[i]])
    mki_0 = statistics.mean(datafinal[datafinal['isFraud'] == 0][key[i]])
    mki_1 = statistics.mean(datafinal[datafinal['isFraud'] == 1][key[i]])
    sigmabetween = math.pow((mki_0 - mk),2) + math.pow((mki_1 - mk),2)
    counter_0 = 0
    counter_1 = 0
    ni = datafinal.shape[0]
    for j in range(datafinal[datafinal['isFraud'] == 0].shape[0]):
        counter_0 += math.pow((datafinal[datafinal['isFraud'] == 0][key[i]].iloc[j] - mki_0),2)
    for k in range(datafinal[datafinal['isFraud'] == 1].shape[0]):    
        counter_1 += math.pow((datafinal[datafinal['isFraud'] == 1][key[i]].iloc[k] - mki_1),2)
    sigmain = 1/ni*(counter_0+counter_1)
    if sigmain == 0:
        sigmain = 0.0001
    fc = sigmabetween/sigmain
    dic[key[i]] = fc
dic
end_time = datetime.datetime.now()
interval = (end_time - start_time).total_seconds()
print ('time used:',interval)

KeyboardInterrupt: 